<a href="https://colab.research.google.com/github/abouslima/AI-Makerspace/blob/master/CloudRun-Deployment/Test-Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Installing required dependencies for the predictions

In [ ]:
!pip install statsmodels==0.12.1
!pip install pycaret==2.3.6
!pip install MarkupSafe==2.0.1
!pip install pickle5

import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


# Loading different models and performing predictions

## PyCaret model for classification

In [ ]:
# downloading the pretrained model from the GitHub repo
!wget -q "https://raw.githubusercontent.com/abouslima/AI-Makerspace/master/CloudRun-Deployment/PyCaret/RF_Model_V1.pkl"

In [ ]:
# loading the pretrained model and creating a function for predictions

from pycaret.regression import load_model, predict_model

model = load_model('RF_Model_V1')

def predict_rating(df):
  predictions_data = predict_model(estimator=model, data=df)
  return predictions_data['Label'][0]

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Transformation Pipeline and Model Successfully Loaded


### Taking the input required for predictions and transforming them into a Pandas DataFrame

In [ ]:
import pandas as pd

Age = 34                    # @param {type:"integer"}
Experience = 9              # @param {type:"integer"}
Income = 150                # @param {type:"integer"}
Family = 1                  # @param {type:"integer"}
CCAvg = 8.9                 # @param {type:"number"}
Education = 3               # @param {type:"integer"}
Mortgage = 0                # @param {type:"integer"}
SecuritiesAccount = 0       # @param {type:"integer"}
CDAccount = 0               # @param {type:"integer"}
Online = 0                  # @param {type:"integer"}
CreditCard = 0              # @param {type:"integer"}

features = {
    'Age': Age,
    'Experience': Experience,
    'Income': Income,
    'Family': Family,
    'CCAvg': CCAvg,
    'Education': Education,
    'Mortgage': Mortgage,
    'SecuritiesAccount': SecuritiesAccount,
    'CDAccount': CDAccount,
    'Online': Online,
    'CreditCard': CreditCard}

features_df = pd.DataFrame([features])

In [ ]:
# predicting the classification class
predict_rating(features_df)

1

## SARIMA Model for forecasting

In [ ]:
# downloading the models from the GitHub Repository 
!wget -q "https://raw.githubusercontent.com/abouslima/AI-Makerspace/master/CloudRun-Deployment/Forecasting/models/Alkoholunfaelle_model.pkl"
!wget -q "https://raw.githubusercontent.com/abouslima/AI-Makerspace/master/CloudRun-Deployment/Forecasting/models/Verkehrsunfaelle_model.pkl"

In [ ]:
# loading the downloaded models and saving them again, because colab runs on python 3.6 which supports pickle4 and the models where saved on python 3.8 using pickle5
import pickle5 as pickle
with open("Verkehrsunfaelle_model.pkl", "rb") as fh:
  data = pickle.load(fh)
  data.save("Verkehrsunfaelle_model.pkl")

with open("Alkoholunfaelle_model.pkl", "rb") as fh:
  data = pickle.load(fh)
  data.save("Alkoholunfaelle_model.pkl")

### Loading the pretrained model and performing the predictions

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAXResults
import numpy as np

n_steps =  10# @param {type:"integer"}

Alkohol_model = SARIMAXResults.load('Alkoholunfaelle_model.pkl')
Verkehrs_model = SARIMAXResults.load('Verkehrsunfaelle_model.pkl')

models = [Alkohol_model, Verkehrs_model]

output_value = []

for model in models:
  forecast = model.get_forecast(steps=n_steps)
  mean_forecast = forecast.predicted_mean
  output_value.append(mean_forecast[-1])

final_value = np.sum(output_value)

print(int(final_value))

3465
